<a href="https://colab.research.google.com/github/MNourMoslem/BioEntCls/blob/master/BiomadicalNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [1]:
!pip install transformers datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=065f4b5797420b9a054042cf84bee15e5c49b0b14ab595cc607772227991d8dd
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
   

##Log To The hub

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Implement The Tokenizer

We going to use DistilBERT base model (uncased) for our problem.
First we are going to define the tokenizer then the model because we need to specify some parameters before and we need to make some preprocessing.

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

checkpoint = "distilbert/distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## Load the Dataset

**BioMed_general_NER**

This dataset consists of manually annotated biomedical abstracts from PubMed, drug descriptions from FDA and abstracts from patents.

huggingface link:
https://huggingface.co/datasets/knowledgator/biomed_NER



### Classes
Here's a description for each of the labels:

* **CHEMICALS** - Represents substances with distinct molecular composition, often involved in various biological or industrial processes.

* **CLINICAL DRUG** - Refers to pharmaceutical substances developed for medical use, aimed at preventing, treating, or managing diseases.

* **BODY SUBSTANCE** - Denotes materials or substances within the human body, including fluids, tissues, and other biological components.

* **ANATOMICAL STRUCTURE** - Describes specific parts or structures within an organism's body, often related to anatomy and physiology.

* **CELLS AND THEIR COMPONENTS** - Encompasses the basic structural and functional units of living organisms, along with their constituent elements.

* **GENE AND GENE PRODUCTS** - Involves genetic information and the resultant products, such as proteins, that play a crucial role in biological processes.

* **INTELLECTUAL PROPERTY** - Pertains to legal rights associated with creations of the mind, including inventions, literary and artistic works, and trademarks.

* **LANGUAGE** - Relates to linguistic elements, including words, phrases, and language constructs, often in the context of communication or analysis.

* **REGULATION OR LAW** - Represents rules, guidelines, or legal frameworks established by authorities to govern behavior, practices, or procedures.

* **GEOGRAPHICAL AREAS** - Refers to specific regions, locations, or places on the Earth's surface, often associated with particular characteristics or significance.

* **ORGANISM** - Denotes a living being, typically a plant, animal, or microorganism, as a distinct biological entity.

* **GROUP** - Encompasses collections of individuals with shared characteristics, interests, or affiliations.

* **PERSON** - Represents an individual human being, often considered as a distinct entity with personal attributes.

* **ORGANIZATION** - Refers to structured entities, institutions, or companies formed for specific purposes or activities.

* **PRODUCT** - Encompasses tangible or intangible items resulting from a process, often associated with manufacturing or creation.

* **LOCATION** - Describes a specific place or position, whether physical or abstract, with potential relevance to various contexts.

* **PHENOTYPE** - Represents the observable characteristics or traits of an organism, resulting from the interaction of its genotype with the environment.

* **DISORDER** - Denotes abnormal conditions or disruptions in the normal functioning of a biological organism, often associated with diseases or medical conditions.

* **SIGNALING MOLECULES** - Involves molecules that transmit signals within and between cells, playing a crucial role in various physiological processes.

* **EVENT** - Describes occurrences or happenings at a specific time and place, often with significance or impact.

* **MEDICAL PROCEDURE** - Involves specific actions or interventions conducted for medical purposes, such as surgeries, diagnostic tests, or therapeutic treatments.

* **ACTIVITY** - Encompasses actions, behaviors, or processes undertaken by individuals, groups, or entities.

* **FUNCTION** - Describes the purpose or role of a biological or mechanical entity, focusing on its intended or inherent activities.

* **MONEY** - Represents currency or financial assets used as a medium of exchange, often in the context of economic transactions.

In [44]:
from datasets import load_dataset

ds = load_dataset("knowledgator/biomed_NER")
ds

DatasetDict({
    train: Dataset({
        features: ['entities', 'text'],
        num_rows: 4840
    })
})

In [45]:
# Usually this is not the best test size to split the data
# but we need more training data to get better results sence we don't have a big dataset.

ds = ds['train'].train_test_split(test_size = 0.1)
ds

DatasetDict({
    train: Dataset({
        features: ['entities', 'text'],
        num_rows: 4356
    })
    test: Dataset({
        features: ['entities', 'text'],
        num_rows: 484
    })
})

## Prepare ids and labels

In [46]:
label2id = {
  'O' : 0,
  'ACTIVITY' : 1,
  'ANATOMICAL STRUCTURE' : 2,
  'BODY SUBSTANCE' : 3,
  'CELLS AND THEIR COMPONENTS' : 4,
  'CHEMICALS' : 5,
  'CLINICAL DRUG' : 6,
  'DISORDER' : 7,
  'EVENT' : 8,
  'FUNCTION' : 9,
  'GENE AND GENE PRODUCTS' : 10,
  'GEOGRAPHICAL AREAS' : 11,
  'GROUP' : 12,
  'INTELLECTUAL PROPERTY' : 13,
  'LANGUAGE' : 14,
  'LOCATION' : 15,
  'MEDICAL PROCEDURE' : 16,
  'MONEY' : 17,
  'ORGANISM' : 18,
  'ORGANIZATION' : 19,
  'PERSON' : 20,
  'PHENOTYPE' : 21,
  'PRODUCT' : 22,
  'REGULATION OR LAW' : 23,
  'SIGNALING MOLECULES': 24,
}

id2label = {v : k for k, v in label2id.items()}

## Preprocess The Data

In [47]:
import torch

# if we look at the data we'll some labeling issues so we are going to fix it manualy
# we could find these label mistakes with a `set()` and a `for loop`.
# and this is the result

replaces = {
  'DISORDERS' : 'DISORDER',
  'EVENTS' : 'EVENT',
  'GENES' : 'GENE AND GENE PRODUCTS',
  'INTELLECTUAL' : 'INTELLECTUAL PROPERTY',
  'LOCATIONS' : 'LOCATION',
  'ORGANISMS' : 'ORGANISM',
  'ORGANISMS ': 'ORGANISM',
  'FINDINGS/PHENOTYPES' : 'PHENOTYPE',
  'ORGANIZATIONS' : 'ORGANIZATION',
  'PRODUCTS': 'PRODUCT'
}


# A function to find an multiple entities inside list of tokens
def find_entities(tokens : torch.tensor, entities : list):
    result = torch.zeros(len(tokens), dtype=torch.int)

    for entity, value in entities:
        sub_len = len(entity)

        for i in range(len(tokens) - sub_len + 1):
            if torch.equal(tokens[i:i + sub_len], entity):
                result[i:i + sub_len] = value

    return result

# The preprocess data function we pass to the dataset `map` function
def preprocess_data(example):
  input_tokens = tokenizer(example['text'].split(), is_split_into_words = True,  padding = 'max_length', truncation = True, return_tensors='pt', max_length=512)

  entities = []
  for entity in example['entities']:
    class_, start, end = entity['class'], entity['start'], entity['end']
    if class_ != 'Unlabelled': # another labeling issue with the dataset
      tokenized_entity = tokenizer.encode(example['text'][start:end].split(), is_split_into_words = True, add_special_tokens=False, return_tensors='pt')
      class_id = label2id[replaces.get(class_, class_)]
      entities.append((tokenized_entity[0], class_id))

  labels = find_entities(input_tokens.input_ids.squeeze(), entities)
  input_tokens['labels'] = labels
  return input_tokens

In [ ]:
t_ds = ds.map(preprocess_data)

Map:   0%|          | 0/4356 [00:00<?, ? examples/s]

In [ ]:
# In this stage we are going to convert the data into batches.
# We would like to to this in the `preprocess_data` function and with the first call of the `map` function, but the dataset is not big so its not a big
# deal to map twice. Also it's easier to read sence this is a tutorial

t_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

def batch_data(example):
  return {
      "batched_input_ids" : example['input_ids'].squeeze(),
      "batched_attention_mask" : example['attention_mask'].squeeze(),
      "batched_labels" : example['labels'],
  }

batched_ds = t_ds.map(batch_data, batched = True, remove_columns=['input_ids', 'attention_mask', 'labels', 'text', 'entities'], batch_size=128)
batched_ds = batched_ds.rename_columns({'batched_input_ids' : 'input_ids', "batched_attention_mask" : "attention_mask", "batched_labels" : "labels"})

## Implement The DataCollator

Data Collators are very helpful classes to deal with batched data. You could read more about it in hugging face page from this link:
https://huggingface.co/docs/transformers/main_classes/data_collator

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer)

## Implement Seqeval Matrix

**Seqeval** is a Python library that is specifically designed for evaluating sequence labeling tasks, such as Named Entity Recognition (NER), Part-of-Speech (POS) tagging, and other tasks where the output is a sequence of labels corresponding to each token in a sentence or document.

In [ ]:
import evaluate

seqeval_matrix = evaluate.load('seqeval')

In [ ]:
import numpy as np

label_list = list(label2id)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval_matrix.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Implement The Model

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(checkpoint, num_labels = len(id2label), id2label = id2label, label2id = label2id)

## Setting Trainer and Training Arguments

In [36]:
from transformers import TrainingArguments, Trainer

model_name = "BioEntCls"

train_args = TrainingArguments(
  output_dir = model_name,
  overwrite_output_dir = True,
  learning_rate = 5e-5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  num_train_epochs=8,
  weight_decay=0.01,
  adam_beta1 = 0.9,
  adam_beta2 = 0.95,
  lr_scheduler_type='cosine',
  eval_strategy = "step",
  save_strategy = "step",
  fp16 = True,
  load_best_model_at_end=True,
  push_to_hub=True,
)

trainer = Trainer(
  model=model,
  args=train_args,
  train_dataset=batched_ds["train"],
  eval_dataset=batched_ds["test"],
  tokenizer=tokenizer,
  data_collator=data_collator,
  compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


## Train The Model

In [37]:
import warnings
warnings.filterwarnings('ignore', category = UserWarning)

trainer.train()

Step,Training Loss
500,0.358400
1000,0.281400


TrainOutput(global_step=1290, training_loss=0.3046902028165122, metrics={'train_runtime': 415.9641, 'train_samples_per_second': 49.451, 'train_steps_per_second': 3.101, 'total_flos': 2688651927582720.0, 'train_loss': 0.3046902028165122, 'epoch': 5.0})

## Push To Hub

In [38]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/MNMoslem/BioEntCls/commit/f178bccbb2ddaabae4f23ec66a56352dd048b23c', commit_message='End of training', commit_description='', oid='f178bccbb2ddaabae4f23ec66a56352dd048b23c', pr_url=None, pr_revision=None, pr_num=None)

##Test The Model

In [39]:
from transformers import pipeline

username = "MNMoslem"
pipe = pipeline(task = 'token-classification', model = f"{username}/{model_name}")

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [40]:
text = "An effective vaccine is urgently needed to prevent the human immunodeficiency virus type 1 (HIV-1) epidemic, a disease transmitted to approximately 1.5 million people each year [1].\nCurrently, lifelong antiretroviral therapy (ART) combinations are the standard of care for the management of HIV in people living with HIV (PLHIV). However, ART does not cure or eradicate the virus from infected cells.\nAntiretroviral therapy efficiently suppresses HIV viral replication, delays disease progression, restores T-cell immune responses, and improves the quality of life and the survival rate of PLHIV [2,3]. Unfortunately, transmissions still occur either from untreated people living with HIV or from PLHIV on antiretroviral therapy with insufficient virological suppression."
print(text)

An effective vaccine is urgently needed to prevent the human immunodeficiency virus type 1 (HIV-1) epidemic, a disease transmitted to approximately 1.5 million people each year [1].
Currently, lifelong antiretroviral therapy (ART) combinations are the standard of care for the management of HIV in people living with HIV (PLHIV). However, ART does not cure or eradicate the virus from infected cells.
Antiretroviral therapy efficiently suppresses HIV viral replication, delays disease progression, restores T-cell immune responses, and improves the quality of life and the survival rate of PLHIV [2,3]. Unfortunately, transmissions still occur either from untreated people living with HIV or from PLHIV on antiretroviral therapy with insufficient virological suppression.


In [43]:
output = pipe(text)

last_start = None
indecies = {}
classes = []

for out in output:
  class_ = out['entity']
  if class_ != "Unlabelled":
    start, end = out['start'], out['end']
    if last_start and indecies[last_start] == start:
      indecies[last_start] = end
    else:
      indecies[start] = end
      last_start = start
      classes.append(class_)

for idx, (k, v) in enumerate(indecies.items()):
  print(f"Word: {text[k:v]: >17} is {classes[idx]}")

Word:           vaccine is PRODUCT
Word:             human is ORGANISM
Word:  immunodeficiency is ORGANISM
Word:             virus is ORGANISM
Word:              type is ORGANISM
Word:                 1 is ORGANISM
Word:            HIV-1) is ORGANISM
Word:         epidemic, is DISORDER
Word:           disease is DISORDER
Word:            people is GROUP
Word:    antiretroviral is MEDICAL PROCEDURE
Word:           therapy is MEDICAL PROCEDURE
Word:              ART) is MEDICAL PROCEDURE
Word:              care is ACTIVITY
Word:        management is ACTIVITY
Word:               HIV is ORGANISM
Word:            people is GROUP
Word:               HIV is ORGANISM
Word:           PLHIV). is ORGANISM
Word:               ART is MEDICAL PROCEDURE
Word:             virus is ORGANISM
Word:          infected is DISORDER
Word:            cells. is CELLS AND THEIR COMPONENTS
Word:    Antiretroviral is MEDICAL PROCEDURE
Word:           therapy is MEDICAL PROCEDURE
Word:               HIV is ORGANISM